In [2]:
# Build CSV file from ../clean_data/*.csv

# Import dependencies
import pandas as pd
#pd.options.display.width = 0
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

import numpy as np
from timezonefinder import TimezoneFinder
from datetime import datetime, timedelta
from tqdm.std import tqdm

from get_weather_data_at_location_and_hour import *

# Import CSV file
file = "../clean_data/Stadium_Data.csv"
df = pd.read_csv(file)

# print(df.head())
# print(len(df))

tf = TimezoneFinder()

df_extended = pd.DataFrame()

for index, row in tqdm( df.iterrows(), total=len(df) ):
    #print(f"Row {index}: {row['ts']}, {row['latitude']}, {row['longitude']}")

    # Convert timestamp to local time
    dt_local = datetime.strptime(row['ts'], '%Y-%m-%dT%H:%M:%S')
    local_time_str = dt_local.strftime('%Y-%m-%dT%H:%M')

    # Figure out local time zone
    local_time_zone_str = tf.timezone_at(lng=row['longitude'], lat=row['latitude'])
    utc_time_str, timezone_diff_hrs = local_time_to_utc_str(local_time_str, local_time_zone_str)

    # Get the elevation at the given latitude and longitude
    elevation_meters = get_elevation_online(row['latitude'], row['longitude'])

    # Get weather data
    get_24hour_data = False # Get weather data for the specified local hour
    weather_data_dict = get_weather_data_at_location_and_hour(row['latitude'], row['longitude'], elevation_meters, utc_time_str, timezone_diff_hrs, get_24hour_data)

    # Convert the weather_data_dict to a pandas Series object
    weather_data_series = pd.Series(weather_data_dict)

    # Append the weather_data_series to the original row
    combined_series = pd.concat([row, weather_data_series], axis=0)

    # Append the combined_row to the new DataFrame
    df_extended = pd.concat([df_extended, combined_series.to_frame().T], axis=0) # to_frame().T converts the Series to a DataFrame row

df_extended.columns = df_extended.columns.str.lower()
print(df_extended)

# Save the DataFrame to a CSV file
df_extended.to_csv("../clean_data_extended/Stadium_Data_Extended.csv", index=False)


  0%|          | 67/29664 [01:22<10:08:10,  1.23s/it]


KeyboardInterrupt: 